In [ ]:
### Import statements that 
import pandas as pd
from auth import jiraAuth
from jira import JIRA
from itertools import tee, islice


In [ ]:
### functions for variable access in json results
def getAggregateTimeSpent(fields):
    return fields['aggregatetimespent']

def getIssuetype(fields):
    return fields['issuetype']['name']

def getTimeSpent(fields):
    return fields['timespent']

def getProjectKey(fields):
    return fields['project']['key']

def getProjectName(fields):
    return fields['project']['name']

def getFixVersion(fields):
    return [fixed['name'] for fixed in fields['fixVersions']] if isinstance(fields['fixVersions'],list)else fields['fixVersions']['name']

def getResolution(fields):
    return '' if fields['resolution'] is None else fields['resolution']['name']

def getPriority(fields):
    return '' if fields['priority'] is None else fields['priority']['name']

def getStatus(fields):
    return fields['status'] if isinstance(fields['status'],str) else fields['status']['name']

def getAssignee(fields):
    return '' if fields['assignee'] is None else fields['assignee']['displayName']

def getCreated(fields):
    return fields['created']

def getCreator(fields):
    return '' if fields['creator'] is None else fields['creator']['displayName']

In [ ]:
### Iterator issues and fields and with some field results

def extractTicketInfo(results):
        data= ((key,
                getIssuetype(fields),
                getTimeSpent(fields),
                getProjectKey(fields),
                getProjectName(fields),
                getFixVersion(fields),
                getAggregateTimeSpent(fields),
                getResolution(fields),
                fields['resolutiondate'],
                fields['workratio'],
                fields['lastViewed'],
                fields['watches']['watchCount'],
                getCreated(fields),
                getPriority(fields),
                fields['labels'],
                fields['timeestimate'],
                fields['aggregatetimeoriginalestimate'],
                fields['issuelinks'],
                getAssignee(fields),
                fields['updated'],
                getStatus(fields),
                [component['name'] for component in fields['components']] if isinstance(fields['components'],list) else fields['components']['name'],
                fields['timeoriginalestimate'],
                fields['description'],
                fields['aggregatetimeestimate'],
                fields['summary'],
                getCreator(fields)) for key, fields in splitKeyAndFields(results))
        return pd.DataFrame(data,columns = ['key','issueType','timeSpent','projectKey','projectName','fixVersion','aggregateTimeSpent','resolution','resulotiondate','workRatio,','lastViewed','watchCount','created','priority','labels','timeEstimate','aggregateTimeOriginalEstimate','issueLink','assignee','updated','status','component','timeOriginalEstimate','description','aggregateTimeEstimate','summary','creator'])

def splitKeyAndFields(resultList):
        for issue in resultList:
            yield issue.key, issue.raw['fields']

In [ ]:
### specifies connection info,  creates a connection. Search for issues and organizes results to a tabular stucture
jiraInfo = {'userName' :jiraAuth['userName'],'password': jiraAuth['password'],'jiraLoc':'https://issues.apache.org/jira','jql':' created > startOfweek(-8) and created<=startOfweek(-7)  ORDER BY key ASC'}
jiraConn = JIRA(server =jiraInfo['jiraLoc'],basic_auth=(jiraInfo['userName'],jiraInfo['password']))
results = jiraConn.search_issues(jiraInfo['jql'],maxResults=False)
fullTickets = extractTicketInfo(results)

In [ ]:
#### functions for pulling from change log

def pullRawChangelog(jiraInfo):
    data = []
    jiraConn= JIRA(server =jiraInfo['jiraLoc'],basic_auth=(jiraInfo['userName'],jiraInfo['password']))
    results = jiraConn.search_issues(jiraInfo['jql'],maxResults=False, expand='changelog')
    for issue in results:
        for history in issue.changelog.histories:
            for item in history.items:
                try:
                    data.append((issue.key, history.created, history.author, item.field, item.toString, item.fromString))
                except:
                    data.append((issue.key, history.created, '', item.field, item.toString, item.fromString))
    return pd.DataFrame(data, columns = ['key', 'created','editor','field','toString','fromString'])


def constructGenericFieldDurations(breadLevels,changeLogField,fieldfunction, fieldName):
    combinedData=pd.concat([breadLevels,changeLogField], axis=0, sort= False).set_index(['key','created'])
    raw =[]
    for _, new_df in combinedData.groupby(level =0):
        rowIter = currentAndNext(new_df.sort_index().drop_duplicates().itertuples())
        for row , nextRow in rowIter:
            index, author , _ = row
            key,timeStart = index
            index2, _, data = nextRow
            _, timeStop = index2
            raw.append((key,timeStart,timeStop,author,data))
    return pd.DataFrame(raw, columns =['key','timeStart','timeStop','author',field])



def currentAndNext(someIterables): 
    current, nexts = tee(someIterables)
    nexts = islice(nexts,1,None)
    return zip(current, nexts)

def firstAndLast(results, fieldFunction):
   def first(key, fields):
       return (key, getCreated(fields),getCreator(fields),'nan')
   def last(key,fields):
       global jiraServerTime
       return (key,jiraServerTime,'nan',fieldFunction(fields))
   return pd.DataFrame((funct(key, fields) for funct in (first,last) for key, fields in splitKeyAndFields(results)), columns = ['key','created','editor','fromString'])



In [ ]:
jiraConn = JIRA(server =jiraInfo['jiraLoc'],basic_auth=(jiraInfo['userName'],jiraInfo['password']))
fieldDict = {'assignee': getAssignee,'priority': getPriority,'status': getStatus}

rawChangeLog= pullRawChangelog(jiraInfo)

In [ ]:
### block of cohensive code to pull change log.
jiraConn = JIRA(server =jiraInfo['jiraLoc'],basic_auth=(jiraInfo['userName'],jiraInfo['password']))
jiraServerTime = jiraConn.server_info()['serverTime']
for field , fieldFunct in fieldDict.items():
    fieldBreadLevels = firstAndLastv1(results,fieldFunct)
    durationFieldData = constructGenericFieldDurations(fieldBreadLevels,rawChangeLog.loc[rawChangeLog['field']==field,['key', 'created','editor','fromString']],fieldFunct,field)
    durationFieldData.to_csv(field+'.csv',index = False)



In [ ]:
fullTickets.to_csv('ticketData.csv')